In [1]:
import pyodbc
import pandas as pd
import datetime as dt
import numpy as np
from datetime import timedelta
from datetime import datetime
from dateutil import parser as dtparser

! pip install pandasql
import pandasql as ps

def dt_form(x):
    return dt.date.isoformat(x)


#Code to connect to data warehouse 1. driver = type of server. 
# server = sql server's name and database = which database on the sql server you want to access.
# trusted connection = yes just means you won't have a security error thrown.
connection = pyodbc.connect(driver = '{SQL SERVER}', 
                            Server = 'shcd-chi-dw1',
                            Database = 'DW_Bass',
                            trusted_connection = 'yes')



In [3]:
# SQL query to create main table for Bass
bassmem = """  \
select fm.customerkey, count(distinct ft.transactionkey) as trans, sum(netsalesamount) as sales
                  , count(distinct case when ic_firsttransactionflag = 0 then transactionkey end) as subtrans
				  , sum(case when ic_firsttransactionflag = 0 then netsalesamount end) as subsales
				  , count(distinct fr.rewardnumber) as rewards
				  , count(distinct case when rewardamountused > 0 then fr.rewardnumber end) as redemps
from fact_member fm
left join fact_transaction ft on fm.customerkey = ft.customerkey
inner join fact_Membership fms on fm.customerkey = fms.customerkey
left join fact_loyaltyreward fr on fms.CustomerMembershipKey = fr.CustomerMembershipKey
where convert(date,enrolldate) >= '2/3/2019'
group by fm.customerkey
order by fm.customerkey """

bass_table = pd.read_sql_query(bassmem,connection)

bass_table.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps
0,4383744,2,184.82,1,-7.00,1,0
1,4385153,0,NaN,0,NaN,0,0
2,4385155,2,185.42,1,108.71,1,0
3,4385156,3,150.12,2,52.62,1,0
4,4385157,1,24.00,0,NaN,0,0
5,4385158,3,361.27,2,5.18,1,0
6,4385160,1,23.90,0,NaN,1,0
7,4385161,1,51.04,0,NaN,0,0
8,4385162,1,25.38,0,NaN,0,0
9,4385163,1,95.98,0,NaN,0,0


In [7]:
# SQL statement to count customers in bass_table and run sql on bass_table dataframe

sub1 = """  \
select count(distinct customerkey) as customers
from bass_table
"""

total_cust= ps.sqldf(sub1,locals())


In [4]:
# SQL statement to count multi-purchase customers in bass_table and run sql on bass_table dataframe

sub2 = """ \
select count(distinct customerkey) as customers, sum(subtrans) as subtrans, sum(subsales) as subsales
from bass_table
where subtrans >= 1
"""

multi=ps.sqldf(sub2,locals())

In [5]:

enr = """ \
select FY_No as year, FP_No as month, count(distinct customerkey) as customers
from fact_member fm
inner join dim_Store ds on fm.EnrollStoreKey = ds.StoreKey
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
where dd.FY_No in (2018, 2019)
and ds.Active = 1
and ds.OpenDate <= '2/4/2018'
group by FY_No, fp_no
order by FY_No, fp_no

"""

enr_mo = pd.read_sql_query(enr,connection)

enr_mo

,year,month,customers
0,2018,1,27911
1,2018,2,37057
2,2018,3,31172
3,2018,4,35812
4,2018,5,49218
5,2018,6,38243
6,2018,7,36684
7,2018,8,43540
8,2018,9,35847
9,2018,10,54995


In [9]:
enr_mo.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\enrbymonth_bass.xlsx')
total_cust.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\totalcust_bass.xlsx')
multi.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\multicust_bass.xlsx')

In [10]:
# SQL statement enrolled 18 purchased in 19

sqlenr2018pur19 = """ \
select FY_No as year, FP_No as month, count(distinct fm.customerkey) as customers, COUNT(distinct case when trans19>=1 then aa.customerkey end) as multi
from fact_member fm
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
left join 
   (select customerkey, COUNT(distinct transactionkey) as trans19 
   from fact_Transaction ft 
   inner join dim_date dd on ft.transactiondate = dd.sqldatestamp 
   where dd.FY_No in (2019)
   group by customerkey) aa on fm.CustomerKey = aa.CustomerKey
where dd.FY_No in (2018)
group by FY_No, fp_no
order by FY_No, fp_no

"""

enr2018pur19 = pd.read_sql_query(sqlenr2018pur19,connection)

In [11]:
# SQL statement enrolled 18 purchased in 19 - comp only

sqlenr2018pur19comp = """ \
select FY_No as year, FP_No as month, count(distinct fm.customerkey) as customers, COUNT(distinct case when trans19>=1 then aa.customerkey end) as multi
from fact_member fm
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
inner join dim_Store ds on fm.EnrollStoreKey = ds.StoreKey
left join 
   (select customerkey, COUNT(distinct transactionkey) as trans19 
   from fact_Transaction ft 
   inner join dim_date dd on ft.transactiondate = dd.sqldatestamp 
   where dd.FY_No in (2019)
   group by customerkey) aa on fm.CustomerKey = aa.CustomerKey
where dd.FY_No in (2018)
and ds.Active = 1
and ds.OpenDate <= '2/4/2018'
group by FY_No, fp_no
order by FY_No, fp_no


"""

enr2018pur19comp = pd.read_sql_query(sqlenr2018pur19comp,connection)

In [12]:
enr2018pur19.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\enr2018pur19_bass.xlsx')
enr2018pur19comp.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\enr2018pur19comp_bass.xlsx')